# SkyyRose Brand Voice LLM Training

Fine-tune Qwen2.5-1.5B-Instruct on SkyyRose brand voice dataset.

**Instructions:**
1. Runtime > Change runtime type > T4 GPU
2. Run all cells
3. Model will be pushed to: huggingface.co/damBruh/skyyrose-brand-voice-llm

In [ ]:
# Install dependencies
!pip install -q trl peft transformers datasets accelerate bitsandbytes huggingface_hub

In [ ]:
# Login to HuggingFace (get token from https://huggingface.co/settings/tokens)
from huggingface_hub import login
from google.colab import userdata

# Use Colab secrets or prompt for token
try:
    hf_token = userdata.get('HF_TOKEN')
except:
    hf_token = input("Enter your HuggingFace token: ")

login(token=hf_token)

In [ ]:
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
import torch

# Configuration
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
DATASET_NAME = "damBruh/skyyrose-brand-voice-training"
OUTPUT_DIR = "./skyyrose-brand-voice-llm"
HF_USERNAME = "damBruh"

print(f"Loading dataset: {DATASET_NAME}")
dataset = load_dataset(DATASET_NAME, split="train")
print(f"Dataset loaded: {len(dataset)} examples")

In [ ]:
# Quantization config for efficient training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Training configuration
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    weight_decay=0.01,
    max_grad_norm=1.0,
    logging_steps=1,
    save_strategy="epoch",
    save_total_limit=2,
    gradient_checkpointing=True,
    push_to_hub=True,
    hub_model_id=f"{HF_USERNAME}/skyyrose-brand-voice-llm",
    seed=42,
)

In [ ]:
print("Initializing SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
)

print("Starting training...")
trainer.train()

In [ ]:
print("Saving and pushing to Hub...")
trainer.save_model()
trainer.push_to_hub()

print(f"\n✅ Training complete!")
print(f"Model available at: https://huggingface.co/{HF_USERNAME}/skyyrose-brand-voice-llm")

In [ ]:
# Test the model
from transformers import pipeline

pipe = pipeline("text-generation", model=f"{HF_USERNAME}/skyyrose-brand-voice-llm", device_map="auto")

messages = [
    {"role": "system", "content": "You are SkyyRose's brand voice - a luxury streetwear brand from Oakland, California."},
    {"role": "user", "content": "Write a product description for a limited edition rose gold bomber jacket."}
]

response = pipe(messages, max_new_tokens=200)
print(response[0]['generated_text'][-1]['content'])